# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.113863e+02     1.480949e+00
 * time: 0.5769550800323486
     1     9.761403e+00     8.917134e-01
 * time: 1.4879381656646729
     2    -1.154806e+01     1.040526e+00
 * time: 1.924795150756836
     3    -3.418577e+01     7.356843e-01
 * time: 2.6000311374664307
     4    -4.732633e+01     5.542582e-01
 * time: 3.2462470531463623
     5    -5.684717e+01     1.991013e-01
 * time: 3.8927621841430664
     6    -5.964455e+01     1.919482e-01
 * time: 4.420489072799683
     7    -6.081992e+01     8.512674e-02
 * time: 4.850468158721924
     8    -6.138238e+01     3.945361e-02
 * time: 5.277125120162964
     9    -6.169248e+01     3.271517e-02
 * time: 5.703619003295898
    10    -6.186096e+01     3.092292e-02
 * time: 6.130918025970459
    11    -6.200060e+01     2.278331e-02
 * time: 6.560705184936523
    12    -6.207489e+01     1.813195e-02
 * time: 7.352417230606079
    13    -6.212205e+01     1.200906e-02
 * time: 7.777557134628296
  

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671061
    AtomicLocal         -18.8557575
    AtomicNonlocal      14.8522565
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485349 
    Xc                  -19.3336813

    total               -62.261666461011